In [1]:
!pip install gradio torch diffusers transformers pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [17]:
import gradio as gr
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import random
from PIL import Image
import os
import re
from collections import defaultdict

# Load Stable Diffusion model - switching to v2-1-base which sometimes works better for abstract concepts
model_id = "stabilityai/stable-diffusion-2-1-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize pipeline with optimized parameters
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16 if device == "cuda" else torch.float32)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)

# Optional: Enable attention slicing for lower memory usage
pipe.enable_attention_slicing()

# Industry-specific design knowledge base
INDUSTRY_DESIGN_KNOWLEDGE = {
    "technology": {
        "symbols": ["circuit", "chip", "cloud", "shield", "lock", "code", "network", "connection", "pixel", "binary"],
        "colors": ["blue", "navy", "teal", "silver", "white", "black"],
        "styles": ["Modern", "Geometric", "Abstract"],
        "aesthetics": "clean, minimalist, futuristic, technical, precise, digital"
    },
    "finance": {
        "symbols": ["graph", "chart", "arrow", "shield", "pillar", "building", "coin", "mountain", "bridge"],
        "colors": ["blue", "green", "gold", "navy", "maroon", "black"],
        "styles": ["Modern", "Geometric"],
        "aesthetics": "stable, trustworthy, professional, established, secure"
    },
    "food": {
        "symbols": ["fork", "spoon", "plate", "leaf", "wheat", "fire", "droplet", "heart", "hand"],
        "colors": ["red", "orange", "green", "brown", "yellow"],
        "styles": ["Modern", "Vintage", "Mascot"],
        "aesthetics": "appetizing, fresh, vibrant, warm, inviting, natural"
    },
    "healthcare": {
        "symbols": ["heart", "plus", "leaf", "hand", "shield", "circle", "pulse", "cross"],
        "colors": ["blue", "green", "teal", "white", "purple"],
        "styles": ["Modern", "Abstract"],
        "aesthetics": "clean, caring, professional, nurturing, trustworthy, reliable"
    },
    "environment": {
        "symbols": ["leaf", "tree", "water", "sun", "mountain", "globe", "recycle", "animal", "plant"],
        "colors": ["green", "blue", "brown", "teal", "yellow"],
        "styles": ["Modern", "Abstract", "Geometric"],
        "aesthetics": "natural, organic, sustainable, clean, renewable, harmonious"
    },
    "creative": {
        "symbols": ["brush", "pen", "eye", "star", "light", "camera", "palette", "prism"],
        "colors": ["purple", "pink", "blue", "yellow", "orange", "multicolor"],
        "styles": ["Abstract", "Modern", "Geometric"],
        "aesthetics": "innovative, inspiring, dynamic, expressive, imaginative"
    },
    "education": {
        "symbols": ["book", "pencil", "bulb", "cap", "star", "apple", "shield", "tree"],
        "colors": ["blue", "red", "green", "maroon", "navy", "gold"],
        "styles": ["Modern", "Vintage"],
        "aesthetics": "knowledge, growth, stability, tradition, wisdom, learning"
    },
    "retail": {
        "symbols": ["bag", "tag", "hanger", "store", "star", "ribbon", "gift", "cart"],
        "colors": ["red", "black", "gold", "purple", "pink"],
        "styles": ["Modern", "Vintage"],
        "aesthetics": "exciting, quality, premium, fashionable, accessible"
    },
    "sports": {
        "symbols": ["ball", "equipment", "mountain", "wave", "flame", "star", "shield"],
        "colors": ["red", "blue", "black", "green", "orange"],
        "styles": ["Modern", "Mascot", "Geometric"],
        "aesthetics": "dynamic, energetic, powerful, competitive, active"
    }
}

# Color harmony database (complementary and analogous color combinations)
COLOR_HARMONIES = {
    "blue": ["navy", "light blue", "teal", "orange", "gold", "white"],
    "red": ["maroon", "pink", "orange", "black", "white", "gold"],
    "green": ["teal", "lime", "forest green", "olive", "blue", "brown"],
    "yellow": ["gold", "orange", "amber", "brown", "black", "blue"],
    "purple": ["lavender", "violet", "magenta", "teal", "gold", "silver"],
    "orange": ["amber", "coral", "peach", "blue", "navy", "brown"],
    "black": ["white", "gray", "silver", "red", "gold", "blue"],
    "white": ["black", "gray", "navy", "red", "blue", "gold"],
    "brown": ["beige", "tan", "cream", "teal", "blue", "green"],
    "pink": ["magenta", "hot pink", "purple", "gray", "white", "teal"],
    "gold": ["black", "navy", "maroon", "purple", "brown", "white"],
    "silver": ["gray", "blue", "purple", "black", "red", "white"]
}

def extract_key_concepts(description):
    """Extract key concepts from description to emphasize in prompt with enhanced NLP."""
    # Split the description into words
    words = description.lower().replace(',', ' ').replace('.', ' ').split()

    # Expanded stop words list for more precise extraction
    stop_words = {'a', 'an', 'the', 'and', 'or', 'but', 'for', 'with', 'in', 'on', 'at', 'to', 'of', 'by',
                 'this', 'that', 'these', 'those', 'from', 'as', 'is', 'are', 'was', 'were', 'be', 'been',
                 'being', 'have', 'has', 'had', 'do', 'does', 'did', 'can', 'could', 'will', 'would', 'should',
                 'may', 'might', 'must', 'am', 'focus', 'concept', 'based', 'about', 'like', 'company', 'business'}

    # Extract potentially meaningful key words
    key_words = [word for word in words if word not in stop_words and len(word) > 2]

    # Weight multi-word phrases higher (likely more significant descriptors)
    phrases = []
    for i in range(len(words) - 1):
        if words[i] not in stop_words and words[i+1] not in stop_words:
            phrases.append(f"{words[i]} {words[i+1]}")

    # If we have very few key words, lower our standards
    if len(key_words) < 2:
        key_words = [word for word in words if word not in stop_words]

    # Combine and deduplicate
    all_concepts = list(set(key_words + phrases))

    # Return unique key concepts, joined with commas
    return ", ".join(all_concepts[:6])  # Limit to top 6 concepts to avoid dilution

def detect_industry(description):
    """Intelligently detect the most likely industry based on the description."""
    description = description.lower()
    scores = defaultdict(int)

    # Define industry keywords for improved detection
    industry_keywords = {
        "technology": ["tech", "software", "app", "digital", "computer", "cyber", "data", "online", "web", "cloud", "ai", "code", "programming"],
        "finance": ["finance", "bank", "invest", "money", "capital", "wealth", "financial", "insurance", "accounting", "broker", "trading"],
        "food": ["food", "restaurant", "cafe", "coffee", "bakery", "organic", "catering", "kitchen", "chef", "meal", "drink", "brewery", "roastery"],
        "healthcare": ["health", "medical", "care", "wellness", "therapy", "clinic", "hospital", "doctor", "patient", "treatment", "pharmacy"],
        "environment": ["environment", "eco", "green", "sustainable", "nature", "organic", "recycle", "conservation", "renewable", "climate"],
        "creative": ["creative", "design", "art", "studio", "media", "film", "photography", "fashion", "music", "agency", "production"],
        "education": ["education", "school", "learn", "teaching", "academy", "training", "tutor", "course", "university", "college", "knowledge"],
        "retail": ["retail", "shop", "store", "fashion", "clothing", "boutique", "market", "goods", "products", "consumer", "shopping", "brand"],
        "sports": ["sport", "fitness", "gym", "athletic", "training", "outdoor", "adventure", "team", "coach", "exercise", "active"]
    }

    # Score each industry based on keyword matches
    for industry, keywords in industry_keywords.items():
        for keyword in keywords:
            if keyword in description:
                scores[industry] += 1

            # Give additional weight to keywords in the beginning of the description
            if any(word.startswith(keyword) for word in description.split()[:3]):
                scores[industry] += 1

    # If we have a clear winner, return it
    if scores:
        return max(scores.items(), key=lambda x: x[1])[0]

    # Default to a general approach if no clear industry is detected
    return "creative"

def suggest_color_scheme(description, industry):
    """Intelligently suggest a harmonious color scheme based on description and industry."""
    description = description.lower()

    # Try to extract color mentions from the description first
    color_words = ["red", "blue", "green", "yellow", "orange", "purple", "black", "white",
                  "pink", "brown", "gray", "silver", "gold", "navy", "teal", "maroon"]

    mentioned_colors = []
    for color in color_words:
        if color in description:
            mentioned_colors.append(color)

    # If user mentioned colors, build scheme around that
    if mentioned_colors:
        primary_color = mentioned_colors[0]
        if len(mentioned_colors) > 1:
            return f"{primary_color} and {mentioned_colors[1]}"
        else:
            # Choose a complementary color from our harmony database
            if primary_color in COLOR_HARMONIES:
                complement = random.choice(COLOR_HARMONIES[primary_color][:3])
                return f"{primary_color} and {complement}"

    # Otherwise suggest based on industry best practices
    if industry in INDUSTRY_DESIGN_KNOWLEDGE:
        colors = INDUSTRY_DESIGN_KNOWLEDGE[industry]["colors"]
        primary = random.choice(colors)
        colors.remove(primary)
        secondary = random.choice(colors)
        return f"{primary} and {secondary}"

    # Fallback to generally versatile color combinations
    default_schemes = [
        "blue and white",
        "navy and gold",
        "teal and gray",
        "black and red",
        "green and blue",
        "purple and silver"
    ]
    return random.choice(default_schemes)

def suggest_style(description, industry):
    """Suggest an appropriate style based on description and industry."""
    description = description.lower()

    # Check for explicit style mentions in description
    style_keywords = {
        "Modern": ["modern", "clean", "simple", "minimal", "flat", "contemporary"],
        "Vintage": ["vintage", "retro", "classic", "old", "traditional", "heritage", "nostalgic"],
        "Abstract": ["abstract", "artistic", "creative", "unique", "fluid", "conceptual"],
        "Geometric": ["geometric", "shapes", "symmetrical", "precision", "mathematical", "structured"],
        "Mascot": ["mascot", "character", "animal", "creature", "person", "friendly", "cartoon"]
    }

    # Score each style based on keyword mentions
    style_scores = {style: 0 for style in style_keywords}
    for style, keywords in style_keywords.items():
        for keyword in keywords:
            if keyword in description:
                style_scores[style] += 1

    # If we have a clear preferred style from the description, use it
    max_score = max(style_scores.values())
    if max_score > 0:
        for style, score in style_scores.items():
            if score == max_score:
                return style

    # Otherwise recommend based on industry conventions
    if industry in INDUSTRY_DESIGN_KNOWLEDGE:
        return random.choice(INDUSTRY_DESIGN_KNOWLEDGE[industry]["styles"])

    # Fallback to Modern as a safe default
    return "Modern"

def expand_minimal_description(description):
    """Expand a minimal description with intelligent inferences to create a more comprehensive prompt."""
    # For very short descriptions, add context based on detected intent
    words = description.strip().split()

    if len(words) <= 3:
        # Detect if this is just a company/brand name or product
        industry = detect_industry(description)

        expansions = []
        # Add industry-specific context
        if industry in INDUSTRY_DESIGN_KNOWLEDGE:
            # Add symbolic references
            symbols = random.sample(INDUSTRY_DESIGN_KNOWLEDGE[industry]["symbols"],
                                   min(3, len(INDUSTRY_DESIGN_KNOWLEDGE[industry]["symbols"])))
            symbol_text = f"incorporating elements of {', '.join(symbols)}"
            expansions.append(symbol_text)

            # Add aesthetic qualities
            aesthetics = INDUSTRY_DESIGN_KNOWLEDGE[industry]["aesthetics"].split(", ")
            aesthetic_text = f"with a {', '.join(random.sample(aesthetics, min(2, len(aesthetics))))} feeling"
            expansions.append(aesthetic_text)

        # Construct an expanded description
        expanded = f"{description} - a {industry} brand {' '.join(expansions)}"
        return expanded

    return description

def generate_logo_prompt(description, style, colors):
    """Generate a multi-part prompt structure with advanced context understanding."""

    # First expand minimal descriptions
    if len(description.strip().split()) <= 5:
        expanded_description = expand_minimal_description(description)
    else:
        expanded_description = description

    # Detect industry for contextual enhancements
    industry = detect_industry(expanded_description)

    # Extract key concepts with improved NLP
    key_concepts = extract_key_concepts(expanded_description)

    # Enhanced style-specific modifiers with concrete visual elements
    style_modifiers = {
        "Modern": "clean lines, flat design, sans-serif aesthetic, negative space, simple geometry, balanced proportions, minimal detailing",
        "Vintage": "retro emblem, badge style, ornate borders, aged texture, classic emblems, heritage feel, worn edges, traditional symbols",
        "Abstract": "non-representational shapes, fluid forms, conceptual elements, asymmetrical composition, artistic expression",
        "Geometric": "perfect symmetry, mathematical precision, grid-based structure, triangles, circles, squares, polygons, golden ratio",
        "Mascot": "character-based, stylized figure with personality, expressive features, dynamic pose, anthropomorphic elements"
    }

    # Industry-specific enhancements if available
    industry_aesthetics = ""
    if industry in INDUSTRY_DESIGN_KNOWLEDGE:
        industry_aesthetics = f", {INDUSTRY_DESIGN_KNOWLEDGE[industry]['aesthetics']}"

    # Color instructions with emphasis
    color_instruction = f"ONLY using the exact colors: {colors}. No other colors."

    # Core representation instructions with expanded context
    representation = (
        f"logo design representing '{expanded_description}'. "
        f"Must visually symbolize: {key_concepts}. "
    )

    # Create a multi-part structured prompt with advanced context
    prompt = (
        f"Professional vector logo, {representation} "
        f"Style: {style}. {style_modifiers.get(style, '')}{industry_aesthetics}. "
        f"{color_instruction} "
        "Professional branding, iconic symbol, centered composition, "
        "ultra high quality, vector graphic, simple background, logo isolated on plain background, "
        "minimalist, high contrast, scalable, brand identity design. "
        "NO text, NO lettering, symbol only. "
        "Professional graphic design, award-winning logo, "
        f"visual representation of {key_concepts}."
    )

    return prompt

def enhance_negative_prompt():
    """Create a comprehensive negative prompt focused on eliminating text and non-logo elements."""
    return (
        "text, words, letters, fonts, lettering, typography, writing, wordmark, "
        "signature, watermark, label, realistic, photograph, photorealistic, "
        "3d render, drawing, illustration, painting, artwork, sketch, doodle, "
        "multiple concepts, collection, website, UI elements, webpage layout, "
        "busy design, cluttered, noisy, low-quality, pixelated, blurry, grainy, "
        "people, faces, animals (unless specifically requested in description), "
        "complex background, detailed background, textured background, gradient background"
    )

def process_image_for_logo(image):
    """Process the generated image to make it more logo-like."""
    # Convert to RGBA to handle transparency
    image = image.convert("RGBA")

    # Here you could add more processing:
    # - Increase contrast
    # - Simplify colors
    # - Add transparency to background
    # - Center the main elements

    return image

def generate_logo(description, style=None, colors=None, seed=None, num_logos=1):
    """Generate logos with smart defaults and adaptive expansion for minimal input."""
    if not description:
        return "Error: Description cannot be empty."

    # Detect industry for contextual enhancements
    detected_industry = detect_industry(description)

    # Apply smart defaults if parameters are unspecified
    if style is None or style == "":
        style = suggest_style(description, detected_industry)
        print(f"Auto-selected style: {style}")

    if colors is None or colors == "":
        colors = suggest_color_scheme(description, detected_industry)
        print(f"Auto-selected colors: {colors}")

    # Convert seed to integer or generate random seed
    if seed is not None and seed.strip():
        try:
            seed_int = int(seed)
        except ValueError:
            seed_int = random.randint(0, 2147483647)
    else:
        seed_int = random.randint(0, 2147483647)

    # Generate intelligently enhanced prompt
    prompt = generate_logo_prompt(description, style, colors)
    negative_prompt = enhance_negative_prompt()

    # Print the enhanced prompt for debugging
    print(f"Enhanced prompt: {prompt}")

    # Create output directory
    os.makedirs("generated_logos", exist_ok=True)

    images = []
    for i in range(num_logos):
        # Use different seeds for multiple logos
        current_seed = seed_int + i
        generator = torch.manual_seed(current_seed)

        # Generate with optimized parameters for logos
        result = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=60,     # More steps for detail
            guidance_scale=9.0,         # More balanced between prompt and creativity
            width=512,
            height=512,
            generator=generator
        )

        # Process the image
        image = process_image_for_logo(result.images[0])

        # Save with descriptive filename including seed for reproducibility
        safe_desc = "".join(c if c.isalnum() else "_" for c in description[:20])
        img_path = f"generated_logos/logo_{safe_desc}_{current_seed}.png"
        image.save(img_path)
        images.append(img_path)

        # Print seed for user reference (helpful for debugging)
        print(f"Generated logo with seed: {current_seed}")

    return images

def logocraft_ui(description, style, colors, random_seed, num_logos):
    # Handle empty inputs with smart defaults
    if not style:
        style = None  # Will trigger auto-selection
    if not colors:
        colors = None  # Will trigger auto-selection

    return generate_logo(description, style, colors, random_seed, int(num_logos))

def suggest_style_for_ui(description):
    """Provide dynamic style suggestion for the UI based on input."""
    if not description:
        return gr.update(value="Modern")

    industry = detect_industry(description)
    suggested_style = suggest_style(description, industry)
    return gr.update(value=suggested_style)

def suggest_colors_for_ui(description):
    """Provide dynamic color suggestion for the UI based on input."""
    if not description:
        return gr.update(value="")

    industry = detect_industry(description)
    suggested_colors = suggest_color_scheme(description, industry)
    return gr.update(value=suggested_colors)

# Enhanced Gradio UI with smart suggestions and dynamic defaults
demo = gr.Blocks()
with demo:
    gr.Markdown("# LogoCraft: AI-Powered Logo Generation")
    gr.Markdown("### Create professional logos with minimal input - our AI fills in the details")

    with gr.Row():
        with gr.Column(scale=2):
            description = gr.Textbox(
                label="Brand Description",
                placeholder="e.g., Ocean conservation nonprofit or simply 'Green Leaf Cafe'",
                info="give a detailed description for better result."
            )
        with gr.Column(scale=1):
            style = gr.Dropdown(["Modern", "Vintage", "Abstract", "Geometric", "Mascot"], label="Logo Style")
        with gr.Column(scale=1):
            colors = gr.Textbox(
                label="Color Scheme",
                placeholder="Auto-suggest based on industry",
                info="Leave blank for AI-suggested colors"
            )

    with gr.Row():
        with gr.Column(scale=1):
            random_seed = gr.Textbox(
                label="Seed",
                placeholder="Enter a number or leave blank",
                info="Save this number to reproduce logos you like"
            )
        with gr.Column(scale=1):
            num_logos = gr.Slider(
                1, 4, step=1,
                label="Number of Variations",
                value=2
            )

    with gr.Row():
        suggest_btn = gr.Button("Suggest Style & Colors", variant="secondary")
        generate_btn = gr.Button("Generate Logos", variant="primary", size="lg")

    # Connect suggestion button to update style and colors
    suggest_btn.click(
        suggest_style_for_ui,
        inputs=description,
        outputs=style
    ).then(
        suggest_colors_for_ui,
        inputs=description,
        outputs=colors
    )

    # Example gallery to show input/output pairs
    gr.Markdown("### Example Prompts (Try these with one click)")

    with gr.Row():
        example1_btn = gr.Button("Tech Startup")
        example2_btn = gr.Button("Coffee Shop")
        example3_btn = gr.Button("Eco Brand")

    # Set up example buttons
    example1_btn.click(
        lambda: [
            gr.update(value="AI security company"),
            gr.update(value=""),
            gr.update(value="")
        ],
        outputs=[description, style, colors]
    )

    example2_btn.click(
        lambda: [
            gr.update(value="Mountain coffee"),
            gr.update(value=""),
            gr.update(value="")
        ],
        outputs=[description, style, colors]
    )

    example3_btn.click(
        lambda: [
            gr.update(value="Sustainable nature products"),
            gr.update(value=""),
            gr.update(value="")
        ],
        outputs=[description, style, colors]
    )

    output_gallery = gr.Gallery(label="Generated Logos", columns=2, height=600)

    gr.Markdown("### How It Works")
    gr.Markdown("""
    1. **Smart Defaults**: Leave style and colors blank, and our AI will suggest appropriate options
    2. **Minimal Input**: Even short descriptions like "Coffee shop" will generate professional logos
    3. **Industry Intelligence**: The system detects your industry and applies relevant design conventions
    4. **Advanced Prompt Engineering**: Your input is expanded with design best practices behind the scenes
    """)

    generate_btn.click(
        logocraft_ui,
        inputs=[description, style, colors, random_seed, num_logos],
        outputs=output_gallery
    )

# Launch the app
demo.launch()

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f203d0ebdbe9fee35d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
